In [1]:
import numpy as np
import pandas as pd
import vtk
import os
from functions import load_monitor_data
from functions import load_monitor_data_for_area
from functions import export_to_vtp
from functions import write_vtp_files_for_area

np.random.seed(23765)

### Set parameters

In [2]:
user = 'Kattelijn'
simulation = 'disable' # 'no-network', 'disable', 'stimulus', 'calcium' 
number_of_neurons = 500
time_steps = np.arange(50000, 150100, 100)
monitor_variables = ['fired', 'fired_fraction', 'electric_activity', 'secondary_variable', 'calcium', 'synaptic_input', 'background_activity', 'grown_axons', 'connected_axons', 'grown_excitatory_dendrites', 'connected_excitatory_dendrites'] # 'fired', 'fired_fraction', 'electric_activity', 'secondary_variable', 'calcium', 'target_calcium', 'synaptic_input', 'background_activity', 'grown_axons', 'connected_axons', 'grown_excitatory_dendrites', 'connected_excitatory_dendrites'
process_by_area = False

### Load positions data

In [3]:
if user == 'Kattelijn':
    base_folder = f'/Volumes/My Passport/viz-{simulation}'
elif user == 'Kevin':
    base_folder = f'/Users/Kevin/Documents/Opleiding/Master/Computational_Science/Year_2/Semester_1/Period_2/Scientific_Visualisation_and_Virtual_Reality/Project/SciVisContest23/SciVisContest23/viz-{simulation}/'

In [ ]:
column_headers_pos = ['local id', 'pos x', 'pos y', 'pos z', 'area', 'type']

df_pos = pd.read_csv(os.path.join(base_folder, 'positions/rank_0_positions.txt'), skiprows= 8, usecols=[0, 1, 2, 3, 4], delimiter=' ', names=column_headers_pos)

df_pos.head()

In [5]:
neurons = np.sort(np.random.choice(np.arange(0, 50000, 1), size=number_of_neurons, replace=False))

df_pos_filtered = df_pos[df_pos['local id'].isin(neurons)]

### Process neurons by area

In [6]:
if process_by_area:
    area_groups = df_pos_filtered.groupby('area')['local id'].apply(list).to_dict()
    
    output_folder = f"{simulation}_by_area_{monitor_variables}_{number_of_neurons}_neurons"
    time_steps_set = set(time_steps)

    for area, neurons_in_area in area_groups.items():
        print(f"Processing {area}")

        df_monitor = load_monitor_data_for_area(area, neurons_in_area, time_steps_set, monitor_variables, base_folder)

        df_merged = pd.merge(df_monitor, df_pos_filtered, left_on='neuron_id', right_on='local id', how='inner')

        write_vtp_files_for_area(df_merged, output_folder, simulation, area, monitor_variables, time_steps)


### Process all neurons at once

In [ ]:
if not process_by_area:
    folder = f"{simulation}_{monitor_variables}_{number_of_neurons}_neurons"
    time_steps_set = set(time_steps)

    df_monitor = load_monitor_data(neurons, time_steps_set, monitor_variables, base_folder)

    for time_point in time_steps:

        df_monitors_at_time = df_monitor[df_monitor['step'] == time_point]

        df_merged = pd.merge(df_pos_filtered, df_monitors_at_time, left_on='local id', right_on='neuron_id')

        filename = f'{simulation}_timestep_{time_point}.vtp'
        export_to_vtp(df_merged.to_dict('records'), folder, filename, monitor_variables)

        print(f"Saved: {os.path.join(folder, filename)}")